In [2]:
import pandas as pd
import polars as pl


In [3]:
import polars.selectors as cs
import re
from datetime import date


In [6]:
!pwd


/home/jovyan


# Carrega a base simples

In [7]:
df = pd.read_csv("~/work/fiap_datathon/data/PEDE_PASSOS_DATASET_FIAP.csv", sep=";")
# checa se o arquivo foi carregado inteiro
assert len(df.index) > 1000


In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1349 entries, 0 to 1348
Data columns (total 69 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   INSTITUICAO_ENSINO_ALUNO_2020  728 non-null    object 
 1   NOME                           1349 non-null   object 
 2   IDADE_ALUNO_2020               728 non-null    object 
 3   ANOS_PM_2020                   728 non-null    object 
 4   FASE_TURMA_2020                728 non-null    object 
 5   PONTO_VIRADA_2020              702 non-null    object 
 6   INDE_2020                      728 non-null    object 
 7   INDE_CONCEITO_2020             728 non-null    object 
 8   PEDRA_2020                     728 non-null    object 
 9   DESTAQUE_IEG_2020              728 non-null    object 
 10  DESTAQUE_IDA_2020              728 non-null    object 
 11  DESTAQUE_IPV_2020              703 non-null    object 
 12  IAA_2020                       728 non-null    o

In [ ]:
df_pl = pl.from_dataframe(df)
assert df_pl.is_empty != False
assert len(df_pl.columns) == 69


In [ ]:
df_pl.glimpse()


Rows: 1349
Columns: 69
$ INSTITUICAO_ENSINO_ALUNO_2020 <str> 'Escola Pública', None, 'Escola Pública', 'Escola Pública', 'Escola Pública', None, None, 'Rede Decisão/União', None, 'Escola Pública'
$ NOME                          <str> 'ALUNO-1', 'ALUNO-2', 'ALUNO-3', 'ALUNO-4', 'ALUNO-5', 'ALUNO-6', 'ALUNO-7', 'ALUNO-8', 'ALUNO-9', 'ALUNO-10'
$ IDADE_ALUNO_2020              <str> '11', None, '12', '10', '10', None, None, '14', None, '13'
$ ANOS_PM_2020                  <str> '2', None, '2', '2', '1', None, None, '3', None, '1'
$ FASE_TURMA_2020               <str> '2H', None, '3H', '1D', '2M', None, None, '4L', None, '3B'
$ PONTO_VIRADA_2020             <str> 'Não', None, 'Não', 'Não', 'Não', None, None, 'Sim', None, 'Não'
$ INDE_2020                     <str> '7.883752', None, '7.856389666666666', '5.076252', '8.077085', None, None, '8.381390666666666', None, '5.1618745'
$ INDE_CONCEITO_2020            <str> 'B', None, 'B', 'D', 'B', None, None, 'B', None, 'D'
$ PEDRA_2020             

In [ ]:
df_pl.schema


Schema([('INSTITUICAO_ENSINO_ALUNO_2020', String),
        ('NOME', String),
        ('IDADE_ALUNO_2020', String),
        ('ANOS_PM_2020', String),
        ('FASE_TURMA_2020', String),
        ('PONTO_VIRADA_2020', String),
        ('INDE_2020', String),
        ('INDE_CONCEITO_2020', String),
        ('PEDRA_2020', String),
        ('DESTAQUE_IEG_2020', String),
        ('DESTAQUE_IDA_2020', String),
        ('DESTAQUE_IPV_2020', String),
        ('IAA_2020', String),
        ('IEG_2020', String),
        ('IPS_2020', String),
        ('IDA_2020', String),
        ('IPP_2020', String),
        ('IPV_2020', String),
        ('IAN_2020', String),
        ('FASE_2021', Float64),
        ('TURMA_2021', String),
        ('INSTITUICAO_ENSINO_ALUNO_2021', String),
        ('SINALIZADOR_INGRESSANTE_2021', String),
        ('PEDRA_2021', String),
        ('INDE_2021', String),
        ('IAA_2021', Float64),
        ('IEG_2021', Float64),
        ('IPS_2021', Float64),
        ('IDA_2021', Flo

## Corrige tipos das colunas

In [ ]:
# índices devem ser do tipo float


## Separa os dataframes por ano e atribui uma coluna para marcar o ano

In [ ]:
dfs = []
for y in ["2020", "2021", "2022"]:
    df_new = (
        ((df_pl.select(["NOME", cs.matches(y)]).with_columns(pl.select(cs.matches(y)))))
        .select(pl.all())
        .rename(lambda c: re.sub(r"_\d+$", "", c))
    )
    df_new.insert_column(
        0, pl.Series("ANO", pl.repeat(y, n=len(df_new), dtype=pl.Int32, eager=True))
    )

    dfs.append(df_new)

dfs


[shape: (1_349, 20)
 ┌──────┬────────────┬─────────────┬─────────────┬───┬─────────────┬────────────┬────────────┬──────┐
 │ ANO  ┆ NOME       ┆ INSTITUICAO ┆ IDADE_ALUNO ┆ … ┆ IDA         ┆ IPP        ┆ IPV        ┆ IAN  │
 │ ---  ┆ ---        ┆ _ENSINO_ALU ┆ ---         ┆   ┆ ---         ┆ ---        ┆ ---        ┆ ---  │
 │ i32  ┆ str        ┆ NO          ┆ str         ┆   ┆ str         ┆ str        ┆ str        ┆ str  │
 │      ┆            ┆ ---         ┆             ┆   ┆             ┆            ┆            ┆      │
 │      ┆            ┆ str         ┆             ┆   ┆             ┆            ┆            ┆      │
 ╞══════╪════════════╪═════════════╪═════════════╪═══╪═════════════╪════════════╪════════════╪══════╡
 │ 2020 ┆ ALUNO-1    ┆ Escola      ┆ 11          ┆ … ┆ 7           ┆ 5.9375     ┆ 7.75       ┆ 10   │
 │      ┆            ┆ Pública     ┆             ┆   ┆             ┆            ┆            ┆      │
 │ 2020 ┆ ALUNO-2    ┆ null        ┆ null        ┆ … ┆ null   

## Realiza o empilhamento somente nas colunas coincidentes para uma primeira análise

In [ ]:
[print(f"{df.schema}\n") for df in dfs]


Schema([('ANO', Int32), ('NOME', String), ('INSTITUICAO_ENSINO_ALUNO', String), ('IDADE_ALUNO', String), ('ANOS_PM', String), ('FASE_TURMA', String), ('PONTO_VIRADA', String), ('INDE', String), ('INDE_CONCEITO', String), ('PEDRA', String), ('DESTAQUE_IEG', String), ('DESTAQUE_IDA', String), ('DESTAQUE_IPV', String), ('IAA', String), ('IEG', String), ('IPS', String), ('IDA', String), ('IPP', String), ('IPV', String), ('IAN', String)])

Schema([('ANO', Int32), ('NOME', String), ('FASE', Float64), ('TURMA', String), ('INSTITUICAO_ENSINO_ALUNO', String), ('SINALIZADOR_INGRESSANTE', String), ('PEDRA', String), ('INDE', String), ('IAA', Float64), ('IEG', Float64), ('IPS', Float64), ('IDA', Float64), ('IPP', Float64), ('REC_EQUIPE_1', String), ('REC_EQUIPE_2', String), ('REC_EQUIPE_3', String), ('REC_EQUIPE_4', String), ('PONTO_VIRADA', String), ('IPV', Float64), ('IAN', Float64), ('NIVEL_IDEAL', String), ('DEFASAGEM', Float64)])

Schema([('ANO', Int32), ('NOME', String), ('FASE', Float64), (

[None, None, None]

In [202]:
dfs[0].with_columns(pl.col("IAA").cast(pl.Float64, strict=False))


ANO,NOME,INSTITUICAO_ENSINO_ALUNO,IDADE_ALUNO,ANOS_PM,FASE_TURMA,PONTO_VIRADA,INDE,INDE_CONCEITO,PEDRA,DESTAQUE_IEG,DESTAQUE_IDA,DESTAQUE_IPV,IAA,IEG,IPS,IDA,IPP,IPV,IAN
i32,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,str,str,str
2020,"""ALUNO-1""","""Escola Pública""","""11""","""2""","""2H""","""Não""","""7.883752""","""B""","""Ametista""","""Seu destaque em 2020: A sua bo…","""Ponto a melhorar em 2021: Empe…","""Seu destaque em 2020: A sua bo…",8.50002,"""8.7""","""7.5""","""7""","""5.9375""","""7.75""","""10"""
2020,"""ALUNO-2""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2020,"""ALUNO-3""","""Escola Pública""","""12""","""2""","""3H""","""Não""","""7.856389666666666""","""B""","""Ametista""","""Seu destaque em 2020: A sua bo…","""Ponto a melhorar em 2021: Empe…","""Seu destaque em 2020: A sua bo…",7.91667,"""8.9""","""7.5""","""5.5""","""8.125""","""8.111113333333334""","""10"""
2020,"""ALUNO-4""","""Escola Pública""","""10""","""2""","""1D""","""Não""","""5.076252""","""D""","""Quartzo""","""Ponto a melhorar em 2021: Melh…","""Ponto a melhorar em 2021: Empe…","""Ponto a melhorar em 2021: Inte…",8.00002,"""4.1""","""6.875""","""0""","""7.1875""","""7.75""","""5"""
2020,"""ALUNO-5""","""Escola Pública""","""10""","""1""","""2M""","""Não""","""8.077085""","""B""","""Ametista""","""Seu destaque em 2020: A sua bo…","""Seu destaque em 2020: As suas …","""Seu destaque em 2020: A sua bo…",7.50002,"""8""","""7.5""","""7.5""","""8.4375""","""8.166665""","""10"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2020,"""ALUNO-1345""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2020,"""ALUNO-1346""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2020,"""ALUNO-1347""","""Escola Pública""","""17""","""1""","""7B""","""Não""","""7.158543333333334""","""C""","""Ágata""","""Seu destaque em 2020: A sua bo…","""Ponto a melhorar em 2021: Empe…","""Seu destaque em 2020: A sua bo…",9.16668,"""8.1""","""6.875""","""4.666666666666667""","""7.34375""","""6.333335""","""10"""


In [194]:
pl.concat([dfs[0], dfs[1]], how="diagonal")


SchemaError: type Float64 is incompatible with expected type String

ANO,NOME,INSTITUICAO_ENSINO_ALUNO,IDADE_ALUNO,ANOS_PM,FASE_TURMA,PONTO_VIRADA,INDE,INDE_CONCEITO,PEDRA,DESTAQUE_IEG,DESTAQUE_IDA,DESTAQUE_IPV,IAA,IEG,IPS,IDA,IPP,IPV,IAN
i32,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
2020,"""ALUNO-1""","""Escola Pública""","""11""","""2""","""2H""","""Não""","""7.883752""","""B""","""Ametista""","""Seu destaque em 2020: A sua bo…","""Ponto a melhorar em 2021: Empe…","""Seu destaque em 2020: A sua bo…","""8.500020000000001""","""8.7""","""7.5""","""7""","""5.9375""","""7.75""","""10"""
2020,"""ALUNO-2""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2020,"""ALUNO-3""","""Escola Pública""","""12""","""2""","""3H""","""Não""","""7.856389666666666""","""B""","""Ametista""","""Seu destaque em 2020: A sua bo…","""Ponto a melhorar em 2021: Empe…","""Seu destaque em 2020: A sua bo…","""7.91667""","""8.9""","""7.5""","""5.5""","""8.125""","""8.111113333333334""","""10"""
2020,"""ALUNO-4""","""Escola Pública""","""10""","""2""","""1D""","""Não""","""5.076252""","""D""","""Quartzo""","""Ponto a melhorar em 2021: Melh…","""Ponto a melhorar em 2021: Empe…","""Ponto a melhorar em 2021: Inte…","""8.000020000000001""","""4.1""","""6.875""","""0""","""7.1875""","""7.75""","""5"""
2020,"""ALUNO-5""","""Escola Pública""","""10""","""1""","""2M""","""Não""","""8.077085""","""B""","""Ametista""","""Seu destaque em 2020: A sua bo…","""Seu destaque em 2020: As suas …","""Seu destaque em 2020: A sua bo…","""7.50002""","""8""","""7.5""","""7.5""","""8.4375""","""8.166665""","""10"""


In [ ]:
df
